<a href="https://colab.research.google.com/github/af001/Northwestern-MSDS/blob/master/MSDS462/Week5/monkeys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [0]:
'''
@author      :  Anton
@description :  Using a pre-trained model, use transfer learning to 
                train a model for a custom task. This project uses
                ResNet50 to identify 10 classes of monkeys. 
@dataset     :  https://www.kaggle.com/slothkong/10-monkey-species/data
'''

#---------------------------------------------------------------
# IMPORTS
#---------------------------------------------------------------
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model, load_model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import os.path

#---------------------------------------------------------------
# GLOBAL VARIABLES
#---------------------------------------------------------------

NUM_EPOCHS = 10
BATCH_SIZE = 8
NUM_TRAIN_IMAGES = 1370
LEARNING_RATE=0.00001
HEIGHT = 300
WIDTH = 300
TRAIN_DIR = "final-dataset"
BATCH_SIZE = 8
FC_LAYERS = [256, 256]
DROPOUT = 0.5

#CLASS_LIST = ['H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9']

# Dataset contains 10 classes of monkey
CLASS_LIST = ['Mantled Howler', 'Patas Monkey', 'Bald Uakari', 'Japanese Macaque',
              'Pygmy Marmoset', 'White Headed Capuchin', 'Silvery Marmoset', 
              'Common Squirrel Monkey', 'Black headed Night Monkey','Nilgiri Langur']

# Test images for single image predictions
IMAGE_LIST = ['test_images/black_headed_night_monkey.jpg', 
              'test_images/japanese_macaque.jpg', 'test_images/patas_monkey.jpg']

#---------------------------------------------------------------
# FUNCTIONS
#---------------------------------------------------------------

'''Create final model for transfer learning. Freeze base layer, add FC and Dropout'''
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

'''Load and prepare the image'''
def load_image(filename):
    # Load the image
    img = load_img(filename, target_size=(300, 300))
    # Convert to array
    img = img_to_array(img)
    # Reshape into a single sample with 3 channels
    img = img.reshape(1, 300, 300, 3)
    # Center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

'''Load an image and predict the class'''
def run_predictions():
    # Load model
    model = load_model('checkpoints/ResNet50_model_weights.h5')
    
    # Iterate images and predict
    for image in IMAGE_LIST:
        img = load_image(image)
        # Predict the class
        result = model.predict(img)
        name = max(result[0])
        pos = [i for i, j in enumerate(result[0]) if j == name]
        print('[+] Image Predictions for: {}'.format(image))
        print('  > Prediction:  {}'.format(CLASS_LIST[pos[0]]))
        print('  > Probability: {}\n'.format(name))
        
#---------------------------------------------------------------
# START
#---------------------------------------------------------------

# Entry point, if the weights exist in checkpoints, run predictions. 
# Else, train and run predictions
if os.path.isfile('checkpoints/ResNet50_model_weights.h5'):
    print('[+] Found weights. Running predictions...\n')
    run_predictions()
else:
    print('[!] No weights found. Training...')
    # Import ResNet50 model and discard FC layers
    base_model = ResNet50(weights='imagenet', 
                          include_top=False, 
                          input_shape=(HEIGHT, WIDTH, 3))

    # Create data generator to get training images from folders. Augment data
    # using flip and rotate to increase accuracy
    train_datagen =  ImageDataGenerator(preprocessing_function=preprocess_input,
          rotation_range=90,
          horizontal_flip=True,
          vertical_flip=True
        )

    train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                        target_size=(HEIGHT, WIDTH), 
                                                        batch_size=BATCH_SIZE)

    # Create final model
    finetune_model = build_finetune_model(base_model, 
                                          dropout=DROPOUT, 
                                          fc_layers=FC_LAYERS, 
                                          num_classes=len(CLASS_LIST))

    # Use Adam optimizer with low learning rate due to fine tuning
    adam = Adam(lr=LEARNING_RATE)
    finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

    # Set checkpoints to save weights
    filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
    checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
    callbacks_list = [checkpoint]

    # Apply fit_generator and set number of epochs, batch size, and data shuffling
    history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                           steps_per_epoch=NUM_TRAIN_IMAGES // BATCH_SIZE, 
                                           shuffle=True, callbacks=callbacks_list)
    print('\n[+] Running predictions...\n')
    run_predictions()

[!] No weights found. Training...
Found 1370 images belonging to 10 classes.
Epoch 1/10
171/171 [==============================] - 36s 209ms/step - loss: 2.8937 - accuracy: 0.1564

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 2/10
171/171 [==============================] - 35s 206ms/step - loss: 2.2621 - accuracy: 0.2012

Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 3/10
171/171 [==============================] - 36s 209ms/step - loss: 2.0777 - accuracy: 0.2584s - loss: 2.0

Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 4/10
171/171 [==============================] - 35s 207ms/step - loss: 1.9851 - accuracy: 0.3164

Epoch 00004: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 5/10
171/171 [==============================] - 35s 205ms/step - loss: 1.7872 - accuracy: 0.3737

Epoch 00005: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 6/10
171/171 [=========================